In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
import math
import time
from tqdm import tqdm

In [12]:
show_detection = 0

In [4]:
data_path = '../data/CALC_STRAIGHT/'
calcSEG = [f for f in os.listdir(data_path) if f.endswith('.csv')]
output_path = '../data/CALC_STRAIGHT_CLEAN/'

In [5]:
X = pd.read_csv('../data/PN chuze/PN ataxie.csv')
X.head()

,File,Ts [s],fs [Hz],Frames,Type,od,do,od.1,do.1,od.2,...,od.6,do.6,od.7,do.7,od.8,do.8,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,N-01-1-Char00,0.008,125.0,30060.0,norma,38314,40368,41497.0,69503.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A-03-1-Char00,0.008,125.0,10482.0,ataxie,40200,41032,42822.0,46285.0,47286.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N-04-1-Char00,0.008,125.0,24964.0,norma,28500,30879,32283.0,34970.0,36292.0,...,52700.0,61820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N-06-1-Char00,0.008,125.0,23941.0,norma,30120,32045,34437.0,36508.0,38509.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N-07-1-Char00,0.008,125.0,34820.0,norma,28900,63720,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
def mean_xy(df):
    avg = pd.DataFrame()
    for dim in ['x','y']:
        dim_cols = [s for s in df.columns.to_list() if dim in s]
        dim_df = df.filter(dim_cols,axis=1)
        avg[dim] = np.mean(dim_df,axis=1)
    return avg

In [7]:
def angle(df):
    theta = np.zeros((np.size(df,axis=0)))
    dx = np.diff(df.x)
    dy = np.diff(df.y)
    for i in range(len(dx)): 
        theta[i] = math.atan2(dy[i], dx[i]) * 180 / math.pi 
    return theta

In [8]:
def exclude(df,T_sampling,low,high,low2,th):
    ex = np.zeros(len(df))

    df['contactL_clean']=df.contactL
    df['contactR_clean']=df.contactR

    CR = np.where(abs(np.diff(df.contactR))>0)[0]
    CL = np.where(abs(np.diff(df.contactL))>0)[0]
    exR_low = np.where(np.diff(CR)*T_sampling < low)[0]
    exL_low = np.where(np.diff(CL)*T_sampling < low)[0]

    stop = [x + 1 for x in exR_low]
    for i,e in enumerate(exR_low):
        #ex[CR[e]:CR[stop[i]]] = 1
        df.loc[CR[e]+1:CR[stop[i]],'contactR_clean'] = abs(df.loc[CR[e]+1:CR[stop[i]],'contactR']-1)

    stop = [x + 1 for x in exL_low]
    for i,e in enumerate(exL_low):
        #ex[CL[e]:CL[stop[i]]] = 1
        df.loc[CL[e]+1:CL[stop[i]],'contactL_clean'] = abs(df.loc[CL[e]+1:CL[stop[i]],'contactL']-1)

    CR = np.where(abs(np.diff(df.contactR_clean))>0)[0]
    CL = np.where(abs(np.diff(df.contactL_clean))>0)[0]
    exR_low = np.where(np.diff(CR)*T_sampling < low2)[0]
    exL_low = np.where(np.diff(CL)*T_sampling < low2)[0]
    exR_high = np.where(np.diff(CR)*T_sampling > high)[0]
    exL_high = np.where(np.diff(CL)*T_sampling > high)[0]

    stop = [x + 1 for x in exR_low]
    for i,e in enumerate(exR_low):
        ex[CR[e]:CR[stop[i]]] = 1

    stop = [x + 1 for x in exL_low]
    for i,e in enumerate(exL_low):
        ex[CL[e]:CL[stop[i]]] = 1

    stop = [x + 1 for x in exL_high]
    for i,e in enumerate(exL_high):
        ex[CL[e]:CL[stop[i]]] = 1

    stop = [x + 1 for x in exR_high]
    for i,e in enumerate(exR_high):
        ex[CR[e]:CR[stop[i]]] = 1

    ex_ones = np.where(ex==1)[0]
    drop = np.where(np.diff(ex_ones) > 1)[0]
    for d in drop:
        if((ex_ones[d+1] - ex_ones[d])*T_sampling)<th:
            ex[ex_ones[d]:ex_ones[d+2]]=1 
    return ex

In [15]:
segs = pd.DataFrame(columns=['subjectID','type','part','t_sampling','sample_len','time_len'])

for seg in tqdm(calcSEG):
    #seg = 'A-03-1-Char00part4seg-0.csv'
    
    subject =  seg.split(sep='Char00',maxsplit=1)[0]+'Char00'
    part = seg.split(sep='part',maxsplit=1)[1][0]
    segm = seg.split(sep='seg-',maxsplit=1)[1][0]
    #print(subject + ':' + part + ':' + segm )
    
    df = pd.read_csv(data_path + seg)
    df.rename(columns={"Unnamed: 0": "OrigIndex"},inplace=True)
    N = np.size(df,axis=0)
    # Check for the T_sampling from bvh file
    T_sampling = X.loc[X['File'] == subject,'Ts [s]'].values
    # Create time vector for this part
    time = np.linspace(0,N,num=N)*T_sampling

    # Take column with position only
    col_list = df.columns.to_list()
    position_cols = [s for s in col_list if "X" in s]
    xdf = df.filter(position_cols,axis=1)
    
    # AVERAGE FROM ALL POSITIONS IN XY PLANE
    xy_cols = []
    for col in xdf.columns.to_list():
        if 'x' in col:
            xy_cols.append(xdf.columns.to_list().index(col))
        if 'y' in col:
            xy_cols.append(xdf.columns.to_list().index(col))
    xy = xdf.filter(xdf.columns[xy_cols],axis=1)
    avg = mean_xy(xy)  
    direction = angle(avg)
    
    # EXCLUSION DRIVEN BY FOOT CONTACTS
    ex = exclude(df,T_sampling,0.1,2,0.2,1)
    
    if show_detection:
        plt.figure(figsize=(16,9))
        plt.plot(df.contactL)
        plt.plot(df.contactR-1.1)
        plt.plot(df.contactL_clean+1.1)
        plt.plot(df.contactR_clean-2.2)

        plt.plot(ex+2.2)

        fig, ax = plt.subplots(5, 1, figsize=(16,5))
        ax[0].plot(avg.x,avg.y)
        ax[0].set_title('AVERAGE FROM ALL POSITIONS: XY projection')
        ax[0].set_ylabel('y')
        ax[0].set_xlabel('x')

        ax[1].plot((np.diff(avg.x)**2 + np.diff(avg.y)**2)**0.5)
        ax[1].plot(savgol_filter((np.diff(avg.x)**2 + np.diff(avg.y)**2)**0.5,59,2),'r')
        ax[1].set_title('DIFFERENCE')
        ax[1].set_ylabel('diff')
        ax[1].set_xlabel('sample')

        ax[2].plot(direction)
        ax[2].plot(savgol_filter(direction,59,2),'r')
        ax[2].set_title('DIRECTION')
        ax[2].set_ylabel('Direction ')
        ax[2].set_xlabel('sample')

        ax[3].plot(np.diff(savgol_filter((np.diff(avg.x)**2 + np.diff(avg.y)**2)**0.5,59,2)),'k')
        ax[3].plot(np.diff(savgol_filter(direction,59,2)),'m')

        ax[4].plot(ex) 

    N_segs = 0
    if sum(ex)==0:
        N_segs=1
        df.to_csv(output_path + seg[:-4] + '_sub_0.csv',index=False)
    else:
        splits = np.where(np.diff(np.where(ex==0)[0])>1)[0]
        N_segs = 1 + len(splits)
        
    for g in range(N_segs):
        if g == 0:
            if ex[0]==0:
                start = 0
            else:
                start = np.where(ex==0)[0][0]  
        else:
            start = np.where(ex==0)[0][splits[g-1]+1]
        if g == N_segs-1:
            stop = len(ex)
        else:
            stop = np.where(ex==0)[0][splits[g]]
        thisSTRAIGHT = df.loc[start:stop,:]
        thisSTRAIGHT_len = (stop - start)*T_sampling
        
        if stop-start > 59:
            if thisSTRAIGHT_len < 5:
                print('WARNING: segment len <5s : ' + subject + part + 'index between: ' + str(start) + '-' + str(stop))
            thisSTRAIGHT.to_csv(output_path + seg[:-4] + '_sub_' + str(g) +'.csv',index=False)
   
    thisDF = pd.DataFrame(data={'subjectID':subject,'type':X.loc[X['File'] == subject,'Type'],
                    'part':part,'t_sampling':T_sampling,'sample_len':N,'time_len':N*T_sampling,
                    'N_seg':N_segs})

    segs = pd.concat([segs, thisDF],ignore_index=True,sort=False)

  7%|▋         | 16/232 [00:44<10:55,  3.03s/it]

  7%|▋         | 17/232 [00:45<08:43,  2.43s/it]

  8%|▊         | 18/232 [00:45<06:33,  1.84s/it]

  9%|▉         | 22/232 [01:01<11:45,  3.36s/it]

 10%|▉         | 23/232 [01:03<10:27,  3.00s/it]

...

 88%|████████▊ | 205/232 [10:10<01:43,  3.82s/it]

 89%|████████▉ | 206/232 [10:12<01:20,  3.08s/it]

 96%|█████████▌| 223/232 [11:03<00:29,  3.22s/it]

 97%|█████████▋| 224/232 [11:09<00:30,  3.87s/it]

100%|██████████| 232/232 [11:34<00:00,  3.65s/it]


In [16]:
segs

,subjectID,type,part,t_sampling,sample_len,time_len,N_seg
0,N-15-1-Char00,norma,2,0.017,522,8.874,1.0
1,N-15-1-Char00,norma,3,0.017,963,16.371,1.0
2,N-04-1-Char00,norma,7,0.008,2047,16.376,1.0
3,A-736123-Char00,ataxie,1,0.017,1388,23.596,2.0
4,N-15051994-Char00,norma,2,0.008,1751,14.008,1.0
5,N-15051994-Char00,norma,3,0.008,1625,13.000,1.0
6,N-920828-Char00,norma,1,0.017,800,13.600,1.0
7,N-04-1-Char00,norma,7,0.008,2087,16.696,1.0
8,N-15-1-Char00,norma,3,0.017,1009,17.153,1.0
9,N-535525-Char00,norma,1,0.017,932,15.844,2.0


In [17]:
segs.to_csv('../data/SEGMENTS_CLEAN.csv',index=False)